# Web Scraper + MongoDB
Daten werden von der Seite https://www.nasdaq.com/market-activity/earnings gewonnen und in der Datenbank DB_Earnings abgespeichert.

In [1]:
from pymongo import MongoClient
from selenium import webdriver
from datetime import datetime as DateTime
from datetime import date
import time

# Verbindung zur Datenbank

In [2]:
client = MongoClient("mongodb+srv://user1:@datenbankearnings.9f0rr.mongodb.net/?retryWrites=true&w=majority")
db = client.get_database("DB_Earnings")
aktien_tabelle = db.Aktie
data_tabelle = db.Daten

## Web Scraper NASDAQ/Earnings
Dieser Abschnitt implementiert einen Web Scraper, der die Seite NASDAQ/Earnings durch geht und die Daten für jede Aktie sammelt.

In [3]:
def strtoArray(symbol_String):
    csv_array = []
    string = ""
    for i in symbol_String:
        if i == " ":
            csv_array.append(string)
            string = ""
        else:
            string = string + i
    return csv_array

In [4]:
def klick(web,tag):
    button = web.find_element_by_xpath("//button [@data-day = '" + str(tag) + "']")
    button.click()
    time.sleep(5)

In [5]:
def auslesen(filter_true, fitler_mrd,tag):
    driver = webdriver.Safari(executable_path="/usr/bin/safaridriver")
    driver.maximize_window()
    driver.get("https://www.nasdaq.com/market-activity/earnings")
    klick(driver,tag)
    content = driver.find_elements_by_class_name("market-calendar-table__row")
    symbol_key = 0  # 1 -> Pre Market; 2 -> After Hours; 3 -> Time not supplied
    daily_array = []
    j = 2 # falls Daten im voraus gelesen werden sollen
    for symbol in content:
        symbol_string = symbol.text
        symbol_string = symbol_string.replace(" ", "")
        symbol_string = symbol_string.replace("\n", " ")
        symbol_array = strtoArray(symbol_string)
        symbol_array = list(filter(None, symbol_array))
        img_src = symbol.find_element_by_class_name("market-calendar-table__icons").get_attribute("src")
        if "time-pre-market" in img_src:
            symbol_key = 1
        elif "time-after-hours" in img_src:
            symbol_key = 2
        elif "time-not-supplied" in img_src:
            symbol_key = 3
        else:
            print("Fehler - key")
        symbol_array.append(symbol_key)
        symbol_array_wert = symbol_array[2 + j].replace(",", "")
        symbol_array_wert = symbol_array_wert.replace("$", "")
        if (int(symbol_array_wert) > (fitler_mrd * (10 ** 9)) and filter_true):
            symbol_array.append(symbol_array_wert)
            daily_array.append(symbol_array)
    driver.close()
    return daily_array

# Aktien in MongoDB Tabelle schreiben

## Create JSON

In [6]:
def create_json_aktie(aktie):
    aktie_json = {}
    aktie_json["Symbol"] = aktie[0]
    aktie_json["Name"] = aktie[1]
    aktie_json["Marktkapitalisierung"] = int(aktie[len(aktie)-1])
    return aktie_json

In [7]:
def create_json_data(aktie):
    data_json = {}
    data_json["Symbol"] = aktie[0]
    eps = aktie[2].replace("$","")
    eps = eps.replace("(","")
    eps = eps.replace(")","")
    data_json["EPS"] = float(eps)
    data_json["Surprise"] = float(aktie[3])
    data_json["Datum"] = aktie[5]
    return data_json

## Insert/Update Daten und Aktien in DB

In [12]:
def data_to_DB(aktie):
    data_json = create_json_data(aktie)
    if(not bool(data_tabelle.find_one(data_json))):
        data_tabelle.insert_one(data_json)

In [13]:
def daily_list_to_DB(daily_list):
    for aktie in daily_list:
        aktie_json = create_json_aktie(aktie)
        data_to_DB(aktie) 
        print(aktie)
        if(not bool(aktien_tabelle.find_one({"Symbol":aktie_json["Symbol"]}))):
            aktien_tabelle.insert_one(aktie_json)
            print("Insert")
        else:
            symbol = aktie_json.pop("Symbol")
            aktie_json.pop("Name")
            aktien_tabelle.update_one({"Symbol":symbol},{"$set":aktie_json})
            print("Update")
        print("_____________")

# Start

In [17]:
def start_scraping(filter_true, filter_mrd):
    tag = int(DateTime.now().strftime('%d'))
    for i in range(3):
        tag = tag - 1 
        daily_list = auslesen(filter_true,filter_mrd, tag)
        daily_list_to_DB(daily_list)


In [18]:
start_scraping(True,10)

['AAPL', 'AppleInc.', '$0.76', '0', '$1,927,912,208,000', 'Sep/2020', '$0.69', '11', 2, '1927912208000']
Update
_____________
['AMZN', 'Amazon.com,Inc.', '$4.23', '0', '$1,584,203,770,389', 'Sep/2020', '$7.30', '16', 2, '1584203770389']
Update
_____________
['GOOG', 'AlphabetInc.', '$10.12', '0', '$1,031,549,772,113', 'Sep/2020', '$11.39', '14', 2, '1031549772113']
Update
_____________
['GOOGL', 'AlphabetInc.', '$10.12', '0', '$1,027,591,219,758', 'Sep/2020', '$11.39', '14', 2, '1027591219758']
Update
_____________
['FB', 'Facebook,Inc.', '$2.12', '0', '$762,542,530,004', 'Sep/2020', '$1.93', '14', 2, '762542530004']
Update
_____________
['CMCSA', 'ComcastCorporation', '$0.65', '27.45', '$191,768,863,775', 'Sep/2020', '$0.51', '12', 3, '191768863775']
Update
_____________
['SHOP', 'ShopifyInc.', '$0.52', '750', '$125,073,495,436', 'Sep/2020', '($0.08)', '22', 3, '125073495436']
Update
_____________
['SNY', 'Sanofi', '$1.07', '-0.93', '$115,935,164,459', 'Sep/2020', '$1.08', '3', 3, '11

['BSX', 'BostonScientificCorporation', '$0.37', '48', '$53,178,177,409', 'Sep/2020', '$0.25', '11', 3, '53178177409']
Update
_____________
['GD', 'GeneralDynamicsCorporation', '$2.90', '1.75', '$40,371,393,745', 'Sep/2020', '$2.85', '8', 3, '40371393745']
Update
_____________
['CTSH', 'CognizantTechnologySolutionsCorporation', '$0.97', '7.78', '$38,553,316,756', 'Sep/2020', '$0.90', '10', 3, '38553316756']
Update
_____________
['EPD', 'EnterpriseProductsPartnersL.P.', '$0.48', '0', '$38,484,191,659', 'Sep/2020', '$0.48', '9', 3, '38484191659']
Update
_____________
['BX', 'TheBlackstoneGroupInc.', '$0.63', '10.53', '$37,798,484,369', 'Sep/2020', '$0.57', '7', 3, '37798484369']
Update
_____________
['EBAY', 'eBayInc.', '$0.72', '16.13', '$37,268,669,427', 'Sep/2020', '$0.62', '4', 3, '37268669427']
Update
_____________
['TEL', 'TEConnectivityLtd.', '$1.16', '34.88', '$35,083,090,211', 'Sep/2020', '$0.86', '9', 3, '35083090211']
Update
_____________
['ORLY', "O'ReillyAutomotive,Inc.", '$7

['EQR', 'EquityResidential', '$0.77', '-6.1', '$19,098,102,180', 'Sep/2020', '$0.82', '10', 3, '19098102180']
Update
_____________
['MXIM', 'MaximIntegratedProducts,Inc.', '$0.72', '24.14', '$18,389,202,667', 'Sep/2020', '$0.58', '10', 3, '18389202667']
Update
_____________
['QSR', 'RestaurantBrandsInternationalInc.', '$0.68', '9.68', '$17,210,770,846', 'Sep/2020', '$0.62', '10', 3, '17210770846']
Update
_____________
['TRU', 'TransUnion', '$0.77', '11.59', '$16,358,105,000', 'Sep/2020', '$0.69', '6', 3, '16358105000']
Update
_____________
['AKAM', 'AkamaiTechnologies,Inc.', '$1.11', '13.27', '$15,847,453,461', 'Sep/2020', '$0.98', '8', 3, '15847453461']
Update
_____________
['VAR', 'VarianMedicalSystems,Inc.', '$1.12', '14.29', '$15,730,801,002', 'Sep/2020', '$0.98', '4', 3, '15730801002']
Update
_____________
['IEX', 'IDEXCorporation', '$1.40', '16.67', '$14,470,059,868', 'Sep/2020', '$1.20', '6', 3, '14470059868']
Update
_____________
['AVTR', 'Avantor,Inc.', '$0.24', '14.29', '$14,